In [1]:
import pandas as pd
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Path to the folder containing your CSV files
data_path = '/content/drive/MyDrive/WattVision_Preprocessing/data/raw/'


In [3]:

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from google.colab import drive

In [4]:
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/WattVision_Preprocessing/data/raw/'
output_dir = '/content/drive/MyDrive/WattVision_Preprocessing/data/outputs/'

os.makedirs(output_dir, exist_ok=True)

# List all CSV files
csv_files = [file for file in os.listdir(data_path) if file.endswith('.csv')]
print("🔹 CSV files found:", csv_files)

# Read and merge
df_list = [pd.read_csv(os.path.join(data_path, file), low_memory=False) for file in csv_files]
df = pd.concat(df_list, ignore_index=True)
print("✅ Merged shape:", df.shape)

merged_path = os.path.join(output_dir, 'smart_6hour_merged.csv')
df.to_csv(merged_path, index=False)
print(f"✅ Merged file saved to: {merged_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🔹 CSV files found: ['smart_6hour_1.csv', 'smart_6hour_2.csv', 'smart_6hour_3.csv', 'smart_6hour_4.csv', 'smart_6hour_5.csv']
✅ Merged shape: (4441793, 32)
✅ Merged file saved to: /content/drive/MyDrive/WattVision_Preprocessing/data/outputs/smart_6hour_merged.csv


In [5]:
# ====================================================================
# STEP 3: Handle Missing Values
# ====================================================================

print("Handling missing values...")
# Loop through all columns in the DataFrame
for col in df.columns:
    if df[col].dtype == 'object':
        # Fill missing categorical values with the mode
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        # Fill missing numerical values with the median
        df[col].fillna(df[col].median(), inplace=True)

print("Missing values have been handled.")
print(f"Total remaining null values: {df.isnull().sum().sum()}")

Handling missing values...


/tmp/ipython-input-2137735863.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-2137735863.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

Missing values have been handled.
Total remaining null values: 0


In [6]:
# ====================================================================
# STEP 4: Feature Engineering
# ====================================================================

# --- Time-based Features ---
# Combine DATE and TIME into a single datetime column before dropping them
df['datetime'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'], errors='coerce')
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['hour'] = df['datetime'].dt.hour
df.drop(columns=['DATE', 'TIME', 'datetime'], inplace=True)

# --- Create the Target Variable (Un-scaled) ---
# Ensure the import/export columns are numeric before subtraction
imp = pd.to_numeric(df['TOTAL_IMPORT (kWh)'], errors='coerce').fillna(0)
exp = pd.to_numeric(df['TOTAL_EXPORT (kWh)'], errors='coerce').fillna(0)

# Calculate NET_IMPORT_kWh in its original kWh unit
df['NET_IMPORT_kWh'] = imp - exp

# --- Create Additional Ratio Feature ---
df['EXPORT_IMPORT_RATIO'] = exp / imp
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.fillna(0, inplace=True)

print("Feature engineering complete. Target variable 'NET_IMPORT_kWh' is NOT scaled.")
print("\n--- Descriptive Statistics for UN-SCALED 'NET_IMPORT_kWh' ---")
print(df['NET_IMPORT_kWh'].describe())

Feature engineering complete. Target variable 'NET_IMPORT_kWh' is NOT scaled.

--- Descriptive Statistics for UN-SCALED 'NET_IMPORT_kWh' ---
count    4.441793e+06
mean     1.967881e+03
std      1.338486e+04
min     -2.176603e+05
25%      7.113330e+02
50%      1.809573e+03
75%      3.143654e+03
max      4.603319e+05
Name: NET_IMPORT_kWh, dtype: float64


In [7]:
# ====================================================================
# STEP 5: Remove Duplicates
# ====================================================================

initial_rows = df.shape[0]
df.drop_duplicates(inplace=True)
print(f"Removed {initial_rows - df.shape[0]} duplicate rows.")

Removed 792 duplicate rows.


In [8]:
# ====================================================================
# STEP 6: Outlier Removal using IQR
# ====================================================================

# Define columns to check for outliers
columns_to_check_outliers = [
    "TOTAL_IMPORT (kWh)", "TOTAL_IMPORT - PV1 (kWh)",
    "TR1_TOTAL_IMPORT (kWh)", "TR2_TOTAL_IMPORT (kWh)", "TR3_TOTAL_IMPORT (kWh)",
    "TR1_TOTAL_IMPORT - PV1 (kWh)", "TR2_TOTAL_IMPORT - PV1 (kWh)", "TR3_TOTAL_IMPORT - PV1 (kWh)",
    "PHASE_A_CURRENT (A)", "PHASE_A_VOLTAGE (V)", "FREQUENCY (Hz)"
]

# --- FIX: Convert columns to numeric before outlier removal ---
# This ensures that any string values are converted, preventing the TypeError.
# 'errors=coerce' will turn any value that can't be converted into NaN (Not a Number).
print("Forcing outlier check columns to be numeric...")
for col in columns_to_check_outliers:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# The outlier removal function will now correctly handle any NaN values created.
print("Numeric conversion complete. Starting outlier removal...")


# Function to remove outliers using the Interquartile Range (IQR) method
def remove_outliers_iqr(data, columns):
    cleaned_data = data.copy()
    for col in columns:
        # The quantile function automatically ignores NaN values, so no extra handling is needed here.
        Q1 = cleaned_data[col].quantile(0.25)
        Q3 = cleaned_data[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        initial_rows = cleaned_data.shape[0]
        # Filter the data to keep only the rows within the valid bounds
        cleaned_data = cleaned_data[(cleaned_data[col] >= lower_bound) & (cleaned_data[col] <= upper_bound)]
        print(f"\nOutlier removal on '{col}':")
        print(f"  - Removed {initial_rows - cleaned_data.shape[0]} rows.")

    return cleaned_data

# Apply outlier removal to the now fully numeric columns
df_no_outliers = remove_outliers_iqr(df, columns_to_check_outliers)

print(f"\nShape after outlier removal: {df_no_outliers.shape}")

Forcing outlier check columns to be numeric...
Numeric conversion complete. Starting outlier removal...

Outlier removal on 'TOTAL_IMPORT (kWh)':
  - Removed 363299 rows.

Outlier removal on 'TOTAL_IMPORT - PV1 (kWh)':
  - Removed 114330 rows.

Outlier removal on 'TR1_TOTAL_IMPORT (kWh)':
  - Removed 116499 rows.

Outlier removal on 'TR2_TOTAL_IMPORT (kWh)':
  - Removed 140617 rows.

Outlier removal on 'TR3_TOTAL_IMPORT (kWh)':
  - Removed 96962 rows.

Outlier removal on 'TR1_TOTAL_IMPORT - PV1 (kWh)':
  - Removed 29812 rows.

Outlier removal on 'TR2_TOTAL_IMPORT - PV1 (kWh)':
  - Removed 31228 rows.

Outlier removal on 'TR3_TOTAL_IMPORT - PV1 (kWh)':
  - Removed 22740 rows.

Outlier removal on 'PHASE_A_CURRENT (A)':
  - Removed 172057 rows.

Outlier removal on 'PHASE_A_VOLTAGE (V)':
  - Removed 89750 rows.

Outlier removal on 'FREQUENCY (Hz)':
  - Removed 30465 rows.

Shape after outlier removal: (3233242, 35)


In [9]:
# ====================================================================
# STEP 7: Data Conversion and Feature Scaling
# ====================================================================

# Convert boolean-like columns to integers (0 or 1)
df['E_BILLING'] = df['E_BILLING'].astype(int)
df['EXPORT'] = df['EXPORT'].astype(int)

# Define all numerical columns that should be scaled.
# CRUCIALLY, 'NET_IMPORT_kWh' is NOT in this list.
numerical_cols_to_scale = [
    'TOTAL_IMPORT (kWh)', 'TOTAL_EXPORT (kWh)', 'TOTAL_IMPORT - PV1 (kWh)',
    'TOTAL_EXPORT - PV1 (kWh)', 'TR1_TOTAL_IMPORT (kWh)', 'TR2_TOTAL_IMPORT (kWh)',
    'TR3_TOTAL_IMPORT (kWh)', 'TR1_TOTAL_EXPORT (kWh)', 'TR2_TOTAL_EXPORT (kWh)',
    'TR3_TOTAL_EXPORT (kWh)', 'TR1_TOTAL_IMPORT - PV1 (kWh)', 'TR2_TOTAL_IMPORT - PV1 (kWh)',
    'TR3_TOTAL_IMPORT - PV1 (kWh)', 'TR1_TOTAL_EXPORT - PV1 (kWh)', 'TR2_TOTAL_EXPORT - PV1 (kWh)',
    'TR3_TOTAL_EXPORT - PV1 (kWh)', 'PHASE_A_CURRENT (A)', 'PHASE_A_VOLTAGE (V)',
    'PHASE_B_CURRENT (A)', 'PHASE_B_VOLTAGE (V)', 'PHASE_C_CURRENT (A)',
    'PHASE_C_VOLTAGE (V)', 'FREQUENCY (Hz)', 'EXPORT_IMPORT_RATIO'
]

# Ensure we only try to scale columns that actually exist in the DataFrame
existing_cols_to_scale = [col for col in numerical_cols_to_scale if col in df.columns]

# Initialize and apply the StandardScaler
scaler = StandardScaler()
df[existing_cols_to_scale] = scaler.fit_transform(df[existing_cols_to_scale])

print("Numerical features have been scaled. The target variable remains un-scaled.")

Numerical features have been scaled. The target variable remains un-scaled.


In [10]:
# ====================================================================
# STEP 9: Drop Final Irrelevant Columns
# ====================================================================

# Drop identifier columns and other columns that are not useful for a general model
cols_to_drop = [
    'household_ID', 'substation_ID',
    # Mostly-missing phase B & C columns
    'PHASE_B_CURRENT (A)', 'PHASE_B_VOLTAGE (V)',
    'PHASE_C_CURRENT (A)', 'PHASE_C_VOLTAGE (V)'
]

# Drop only if they exist in the DataFrame
existing_cols_to_drop = [col for col in cols_to_drop if col in df.columns]
df.drop(columns=existing_cols_to_drop, inplace=True)

print(f"Dropped final irrelevant columns: {existing_cols_to_drop}")

Dropped final irrelevant columns: ['household_ID', 'substation_ID', 'PHASE_B_CURRENT (A)', 'PHASE_B_VOLTAGE (V)', 'PHASE_C_CURRENT (A)', 'PHASE_C_VOLTAGE (V)']


In [11]:
# ==============================================================
# STEP 9: Save Final Cleaned Dataset
# ==============================================================

final_path = os.path.join(output_dir, 'final_cleaned_smart_6hour.csv')
df.to_csv(final_path, index=False)
print(f"🎯 FINAL CLEANED DATA SAVED → {final_path}")

🎯 FINAL CLEANED DATA SAVED → /content/drive/MyDrive/WattVision_Preprocessing/data/outputs/final_cleaned_smart_6hour.csv
